In [1]:
from tensorflow.keras.preprocessing import image
from keras.models import load_model
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import datetime
from geopy.geocoders import Nominatim
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
import imutils
import numpy as np
from pygame import mixer
import time
from threading import Thread

pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
model = load_model(r'C:\Users\Lenovo\Desktop\Projet\cnn.keras')

In [ ]:
TOTAL_CONSEC = 0
TOTAL_THRESH = 20
FIRE = False

# Fonction pour préparer une image pour la classification
def prepare_image(file):
    img_path = ''  # Spécifiez le chemin si nécessaire
    img = image.load_img(img_path + file, target_size=(224, 224))
    # Convertir en tableau de pixels et redimensionner
    img_array = image.img_to_array(img)
    # Rescaler les pixels entre 0 et 1
    img_array = img_array / 255.0
    # Ajouter une dimension pour correspondre au format attendu par le modèle
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return img_array_expanded_dims

def send_fire_alert():
    # Obtenir l'heure actuelle
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Obtenir la position GPS (exemple avec des coordonnées fixes pour Tétouan)
    geolocator = Nominatim(user_agent="fire_detection_system")
    location = geolocator.reverse("35.6875, -5.9166")  # Remplacez par des coordonnées réelles si disponibles
    address = location.address if location else "Localisation inconnue"
    
    # Construire le message
    alert_message = f"wa l7ari9 l7ri9 3ta9 akhay ayoub  \nLocalisation: {address}\nHeure: {current_time}"
    
    # Configuration de l'e-mail
    from_email = "firedetector192@gmail.com"
    to_email = "anas.elkhatib@etu.uae.ac.ma"
    subject = "Alerte Incendie"
    
    # Créer le message
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(alert_message, 'plain'))
    
    # Se connecter au serveur SMTP et envoyer l'e-mail
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)  # Serveur SMTP pour Gmail
        server.starttls()  # Démarrer le TLS
        server.login(from_email, "biwz vqwf xbna vwfg ")  # Se connecter à l'adresse e-mail
        server.send_message(msg)  # Envoyer l'e-mail
        server.quit()  # Déconnexion
        print("[INFO] Alerte de feu envoyée par e-mail!")
    except Exception as e:
        print(f"[ERROR] Échec de l'envoi de l'e-mail: {str(e)}")

# Classe pour gérer le flux vidéo en temps réel
class VideoStream:
    def __init__(self, src=0, resolution=(320, 240), framerate=32):
        # Initialiser la capture vidéo
        self.stream = cv2.VideoCapture(src)
        self.stream.set(cv2.CAP_PROP_FRAME_WIDTH, resolution[0])
        self.stream.set(cv2.CAP_PROP_FRAME_HEIGHT, resolution[1])
        self.frame = None
        self.stopped = False

    def start(self):
        # Démarrer un nouveau thread pour lire les frames
        Thread(target=self.update, args=()).start()
        return self

    def update(self):
        # Boucle pour lire continuellement les frames jusqu'à l'arrêt
        while True:
            if self.stopped:
                return
            (grabbed, self.frame) = self.stream.read()
            if not grabbed:
                self.stop()

    def read(self):
        # Retourner la frame la plus récente
        return self.frame

    def stop(self):
        # Indiquer que le thread doit s'arrêter
        self.stopped = True
        self.stream.release()

# Initialiser le flux vidéo et permettre au capteur de caméra de se chauffer
print("[INFO] starting video stream...")
vs = VideoStream().start()
time.sleep(2.0)
start = time.time()
f = 0

# Initialiser le mixer en dehors de la boucle pour éviter de le réinitialiser à chaque fois
mixer.init()

# Boucle sur les frames du flux vidéo
while True:
    # Prendre la frame du flux vidéo threadé
    frame = vs.read()
    if frame is None:
        break
    f += 1
    frame = imutils.resize(frame, width=400)

    # Sauvegarder la frame temporairement pour la passer à la fonction prepare_image
    cv2.imwrite("temp_frame.jpg", frame)
    preprocessed_image = prepare_image("temp_frame.jpg")  # Utilisation de la fonction prepare_image mise à jour

    # Classifier l'image d'entrée
    begin = time.time()
    prediction = model.predict(preprocessed_image)[0][0]  # Obtenir la prédiction pour la frame
    terminate = time.time()

    # Si la probabilité est proche de 1 (Fire), sinon Non Fire
    if prediction > 0.8:  # Seuil de 0.5 pour classifier Fire
        label = "Fire"
        proba = prediction  # Probabilité de Fire
        TOTAL_CONSEC += 1
        color = (0, 0, 255)  # Rouge pour le feu

        # Si on a atteint le seuil de consécution, déclencher l'alarme
        if not FIRE and TOTAL_CONSEC >= TOTAL_THRESH:
            FIRE = True
            # send_fire_alert()
            if not mixer.music.get_busy():
                mixer.music.load(r"C:\Users\Lenovo\Desktop\Projet\alarm-sound.mp3")
                mixer.music.play(-1)  # Joue l'alarme en boucle
    else:  # Si la probabilité est inférieure à 0.5 (Non Fire)
        label = "Not Fire"
        proba = 1 - prediction  # Probabilité de Non Fire
        TOTAL_CONSEC = 0
        color = (0, 255, 0)  # Vert pour Non Fire

        if FIRE:
            FIRE = False
            mixer.music.stop()  # Arrêter l'alarme

    # Construire le label et l'afficher sur la frame
    label = "{}: {:.2f}%".format(label, proba * 100)
    frame = cv2.putText(frame, label, (10, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Afficher la frame de sortie
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # Si la touche `q` est pressée, sortir de la boucle
    if key == ord("q"):
        print("[INFO] classification took {:.5f} seconds".format(terminate - begin))
        end = time.time()
        break

# Nettoyage
print("[INFO] cleaning up...")
seconds = end - start
print("Time taken: {0} seconds".format(seconds))
fps = f / seconds
print("Estimated frames per second: {0}".format(fps))

cv2.destroyAllWindows()
vs.stop()s